In [1]:
import azureml.core 
from azureml.core import Workspace


In [13]:
ws = Workspace.from_config(path='../config.json') #get default workspace

In [11]:
'''
import os
# Create a folder for the pipeline step files
experiment_folder = 'diabetes_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)
'''

diabetes_pipeline


In [42]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compuinst = "compuinst"

try:
    print('Check for existing compute target')
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=compuinst)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('If it doesnt already exist, create it')
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, compuinst, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print('exception below:')
        print(ex)
    

Check for existing compute target
If it doesnt already exist, create it
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [43]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env",  "environment.yaml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


In [50]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Dataset

# Get the training dataset
diabetes_ds = Dataset.get_by_name(ws,name="diabetes")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                #source_directory = '.',
                                script_name = "prep_diabetes.py",
                                arguments = ['--input-data', diabetes_ds.as_named_input('diabetes'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = '.',
                                script_name = "train_diabetes.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [51]:
#! pip install azureml.widgets

In [52]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-diabetes-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Pipeline is built.
Created step Prepare Data [6fff886b][9cf2f426-87c7-48fb-83bd-fd54c622e318], (This step will run and generate new outputs)
Created step Train and Register Model [149e15f5][5e2f97c8-f31c-4010-a52a-f96e7f248f0a], (This step will run and generate new outputs)
Submitted PipelineRun 07eb3e7d-b6b6-443d-8864-0358e4298922
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/07eb3e7d-b6b6-443d-8864-0358e4298922?wsid=/subscriptions/d8f2241f-6180-41e3-a1de-149047238a9a/resourcegroups/mlrg/workspaces/mlws&tid=ab6d3f0b-f952-404c-89ea-8fd89b826468
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 07eb3e7d-b6b6-443d-8864-0358e4298922
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/07eb3e7d-b6b6-443d-8864-0358e4298922?wsid=/subscriptions/d8f2241f-6180-41e3-a1de-149047238a9a/resourcegroups/mlrg/workspaces/mlws&tid=ab6d3f0b-f952-404c-89ea-8fd89b826468
PipelineRun Status: Running


StepRunId: dcec9526-0150-4a59-b064-e32dafa63c50
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/dcec9526-0150-4a59-b064-e32dafa63c50?wsid=/subscriptions/d8f2241f-6180-41e3-a1de-149047238a9a/resourcegroups/mlrg/workspaces/mlws&tid=ab6d3f0b-f952-404c-89ea-8fd89b826468
StepRun( Prepare Data ) Status: Running

StepRun(Prepare Data) Execution Summary
StepRun( Prepare Data ) Status: Finished


In [ ]:
for run in pipeline_run.get_children():
    print(run.name, ':')
    metrics = run.get_metrics()
    for metric_name in metrics:
        print('\t',metric_name, ":", metrics[metric_name])

Train and Register Model :
	 Accuracy : 0.7272727272727273
	 AUC : 0.7029178860389051
	 ROC : aml://artifactId/ExperimentRun/dcid.92191006-f79e-4f7a-b5a6-7811fac195c6/ROC_1640076538.png
Prepare Data :
	 raw_rows : 768
	 processed_rows : 768


In [31]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 1
	 Training context : Pipeline
	 AUC : 0.7029178860389051
	 Accuracy : 0.7272727272727273


2012_iris_classify version: 1
	 data-format : CSV


classification_model version: 1
	 data-format : CSV




In [34]:
#publishing the pipeline
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="diabetes-training-pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
diabetes-training-pipeline,0b970f10-c419-4480-bfd8-15a002ea7796,Active,REST Endpoint


In [35]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://centralindia.api.azureml.ms/pipelines/v1.0/subscriptions/cc56adb1-a55d-4d23-b1e2-0c252c8f9c2b/resourceGroups/mlrg/providers/Microsoft.MachineLearningServices/workspaces/mlws/PipelineRuns/PipelineSubmit/0b970f10-c419-4480-bfd8-15a002ea7796


In [36]:
#Call the pipeline endpoint
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

Authentication header ready.


In [37]:
import requests

experiment_name = 'mslearn-diabetes-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

'510b606b-e3de-4a4d-a2f6-3128970d0b0f'

In [38]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
published_pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 510b606b-e3de-4a4d-a2f6-3128970d0b0f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/510b606b-e3de-4a4d-a2f6-3128970d0b0f?wsid=/subscriptions/cc56adb1-a55d-4d23-b1e2-0c252c8f9c2b/resourcegroups/mlrg/workspaces/mlws&tid=b194dafb-5834-49bc-b629-fbe86f2808bd

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '510b606b-e3de-4a4d-a2f6-3128970d0b0f', 'status': 'Completed', 'startTimeUtc': '2021-12-21T09:57:10.611213Z', 'endTimeUtc': '2021-12-21T09:57:12.3601Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'Unavailable', 'runType': 'HTTP', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.pipelineComponent': 'pipelinerun', 'azureml.pipelineid': '0b970f10-c419-4480-bfd8-15a002ea7796'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://mlws5838486744.blob.core.windows.net/azureml/ExperimentRun/dcid.510b606b-e3de-4a4d-a2f6-

'Finished'

# Deploy model as service

In [3]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

diabetes_model version 1


In [4]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

./diabetes_service folder created.


In [9]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig,Model
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment(name='service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
#from azureml.core.webservice import LocalWebservice
#deployment_config = LocalWebservice.deploy_configuration(port=8890)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service1"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-21 16:56:28+05:30 Creating Container Registry if not exists.
2021-12-21 16:56:29+05:30 Use the existing image.
2021-12-21 16:56:29+05:30 Generating deployment configuration.
2021-12-21 16:56:30+05:30 Submitting deployment to compute.
2021-12-21 16:56:40+05:30 Checking the status of deployment diabetes-service1..
2021-12-21 16:58:03+05:30 Checking the status of inference endpoint diabetes-service1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [10]:
print(service.get_logs())

2021-12-21T11:27:51,929262800+00:00 - iot-server/run 
2021-12-21T11:27:51,936070800+00:00 - rsyslog/run 
2021-12-21T11:27:51,949082300+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-12-21T11:27:51,961778500+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-12-21T11:27:52,244159300+00:00 - iot-server/finish 1 0
2021-12-21T11:27:52,247273800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-12-21 11:27:53,230 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-12-21 11:27:53,230 | root | INFO | Starting up request id generator
2021-12-21 11:27:53,230 | root | INFO | Sta